# AutoEncoder CUDA - Quick Start Guide

Run on **GPU runtime** (Runtime -> Change runtime type -> T4 GPU)

## 1. Setup

In [ ]:
# Clone repository
import os

repos = "https://github.com/QuackPhuc/AutoEncoder-CUDA.git"

if not os.path.exists('/content/AutoEncoder-CUDA'):
    !git clone --recursive {repos}

%cd /content/AutoEncoder-CUDA
!chmod +x scripts/*.sh build.sh run.sh

In [ ]:
# Download dataset & Build
!scripts/download_cifar10.sh
!./build.sh --clean

---
## 2. Usage Options

| Command | Description |
|---------|-------------|
| `train-autoencoder` | Train autoencoder only |
| `train-svm` | Train SVM with existing encoder weights |
| `evaluate` | Evaluate with pre-trained weights |
| `pipeline` | Full: train -> SVM -> evaluate (default) |

### Option A: Evaluate with Pre-trained Weights (Fast)

In [ ]:
# Use pre-trained encoder.weights + svm.bin
!./run.sh evaluate

### Option B: Train Autoencoder Only

In [ ]:
# Quick test: 5 epochs, 1000 samples
!./run.sh train-autoencoder --epochs 5 --samples 1000

In [ ]:
# Full training: 20 epochs, all samples (~2 hours)
# !./run.sh train-autoencoder --epochs 20

### Option C: Train SVM (requires trained encoder)

In [ ]:
# Train SVM using default encoder weights
!./run.sh train-svm

# Or specify custom encoder weights:
# !./run.sh train-svm --encoder-weights ./checkpoints/gpu_opt_v2.weights

### Option D: Full Pipeline

In [ ]:
# Train autoencoder -> Train SVM -> Evaluate
!./run.sh pipeline --epochs 20

---
## 3. Benchmark

In [ ]:
# Compare CPU vs GPU versions (quick)
!scripts/benchmark.sh --epochs 3 --samples 100

In [ ]:
# GPU-only benchmark (more samples)
!scripts/benchmark.sh --epochs 3 --samples 10000 --gpu-only

---
## 4. Advanced Options

```bash
# GPU versions: naive (basic), v1 (memory opt), v2 (kernel fusion)
./run.sh train-autoencoder --version v2 --epochs 20

# Custom weight paths
./run.sh evaluate --encoder-weights ./checkpoints/my.weights --svm-model ./checkpoints/my.bin

# CPU training
./run.sh train-autoencoder --device cpu --epochs 5 --samples 100
```

---
## 5. Output Files

| Path | Description |
|------|-------------|
| `checkpoints/encoder.weights` | Trained encoder |
| `checkpoints/svm.bin` | Trained SVM model |
| `results/classification_report.txt` | Accuracy report |
| `results/confusion_matrix.csv` | Confusion matrix |